# 🎥 Video Frame Search System with BLIP & Pinecone

This notebook sets up a complete video semantic search engine that:
- Extracts frames from videos
- Generates captions using BLIP
- Stores embeddings in Pinecone
- Enables natural language search

---

## 📋 Prerequisites
1. **GPU Runtime**: Go to `Runtime` → `Change runtime type` → Select `GPU` (T4 recommended)
2. **Pinecone API Key**: You'll enter this in Step 3

---

## 🔧 Step 1: Setup - Clone Repository & Install Dependencies

This will:
- Clone the GitHub repository
- Install all required packages
- Check GPU availability

**⏱️ Expected time: 3-5 minutes**

In [ ]:
# Clone the repository
!git clone https://github.com/pranavacchu/capstone-BLIP.git
%cd capstone-BLIP

# Install dependencies
print("📦 Installing dependencies... This will take 3-5 minutes")
!pip install -q opencv-python-headless pillow numpy pandas tqdm python-dotenv
!pip install -q torch torchvision transformers sentence-transformers
!pip install -q pinecone FlagEmbedding

print("\n✅ Installation complete!")

# Check GPU availability
import torch
if torch.cuda.is_available():
    print(f"\n🚀 GPU detected: {torch.cuda.get_device_name(0)}")
    print(f"   Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    print("\n⚠️ No GPU detected. Using CPU (slower but works)")

## 🔑 Step 2: Configure Pinecone API Key

Enter your Pinecone credentials below:
- **API Key**: Your Pinecone API key
- **Index Host**: Your index URL (from Pinecone dashboard)

Your current settings:
```
API Key: pcsk_51Fgoo_2S9NQf4CHi8LMpX7AXKv4TEHgRdXR3huZcCwBdJkr7BMvmdGHeRASrk5hkz4AH1
Host: https://capstone-b5a0x4x.svc.aped-4627-b74a.pinecone.io
```

In [ ]:
import os

# Set your Pinecone credentials
PINECONE_API_KEY = "pcsk_51Fgoo_2S9NQf4CHi8LMpX7AXKv4TEHgRdXR3huZcCwBdJkr7BMvmdGHeRASrk5hkz4AH1"
PINECONE_HOST = "https://capstone-b5a0x4x.svc.aped-4627-b74a.pinecone.io"
PINECONE_ENVIRONMENT = "us-east-1"

# Write to .env file
with open('.env', 'w') as f:
    f.write(f"PINECONE_API_KEY={PINECONE_API_KEY}\n")
    f.write(f"PINECONE_HOST={PINECONE_HOST}\n")
    f.write(f"PINECONE_ENVIRONMENT={PINECONE_ENVIRONMENT}\n")

print("✅ Configuration saved!")

## 🧪 Step 3: Test Connection to Pinecone

Let's verify the system can connect to your Pinecone database.

In [ ]:
from video_search_engine import VideoSearchEngine

print("🔌 Connecting to Pinecone...")
engine = VideoSearchEngine()

# Get database stats
stats = engine.get_index_stats()

print("\n✅ Successfully connected to Pinecone!")
print(f"\n📊 Database Statistics:")
print(f"   Index: capstone")
print(f"   Total vectors: {stats.get('total_vectors', 0):,}")
print(f"   Dimension: {stats.get('dimension', 1024)}")
print(f"   Capacity: Serverless")

## 📤 Step 4: Upload a Video File

You have two options:

### Option A: Upload from your computer
Run the cell below and click "Choose Files" to upload a video.

### Option B: Use a sample video URL
Download a sample video directly from a URL.

In [ ]:
from google.colab import files
import os

print("📤 Choose how to get your video:\n")
print("1. Upload from computer (recommended for small files < 100MB)")
print("2. Download from URL")
print("3. Use sample video\n")

choice = input("Enter choice (1/2/3): ")

if choice == "1":
    print("\n📁 Please select your video file...")
    uploaded = files.upload()
    video_path = list(uploaded.keys())[0]
    print(f"✅ Uploaded: {video_path}")
    
elif choice == "2":
    video_url = input("Enter video URL: ")
    video_filename = "downloaded_video.mp4"
    print(f"⬇️ Downloading from {video_url}...")
    !wget -O {video_filename} "{video_url}"
    video_path = video_filename
    print(f"✅ Downloaded: {video_path}")
    
else:
    # Use a sample video (you can add a default URL here)
    print("⚠️ No sample video configured. Please choose option 1 or 2.")
    video_path = None

if video_path and os.path.exists(video_path):
    file_size = os.path.getsize(video_path) / (1024*1024)  # MB
    print(f"\n📹 Video ready: {video_path} ({file_size:.1f} MB)")
else:
    print("\n❌ No video file available. Please run this cell again.")

## 🎬 Step 5: Process the Video

This will:
1. Extract frames from the video (removing redundant frames)
2. Generate captions using BLIP AI model
3. Create embeddings for semantic search
4. Upload to Pinecone database

**⏱️ Expected time:**
- 1 minute video: ~2-3 minutes with GPU
- 5 minute video: ~8-10 minutes with GPU
- CPU mode: 3-5x slower

In [ ]:
import time
from datetime import datetime

if 'video_path' not in locals() or not video_path:
    print("❌ Please upload a video first (run the previous cell)")
else:
    # Set video name
    video_name = input("Enter a name for this video (or press Enter for auto-name): ").strip()
    if not video_name:
        video_name = f"video_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
    
    print(f"\n🎬 Processing video: {video_name}")
    print("⏳ This will take a few minutes... Please wait.\n")
    print("=" * 60)
    
    start_time = time.time()
    
    try:
        # Process the video
        stats = engine.process_video(
            video_path=video_path,
            video_name=video_name,
            save_frames=False,  # Set to True to save frames
            upload_to_pinecone=True
        )
        
        processing_time = time.time() - start_time
        
        print("\n" + "=" * 60)
        print("\n✅ VIDEO PROCESSING COMPLETE!\n")
        print(f"📊 Processing Statistics:")
        print(f"   Video name: {video_name}")
        print(f"   Frames extracted: {stats['total_frames_extracted']:,}")
        print(f"   Captions generated: {stats['frames_with_captions']:,}")
        print(f"   Embeddings uploaded: {stats['embeddings_uploaded']:,}")
        print(f"   Processing time: {processing_time/60:.1f} minutes")
        print(f"\n   Frame reduction: {(1 - stats['frames_with_captions']/stats['total_frames_extracted'])*100:.1f}%")
        
        # Save video_name for next steps
        processed_video_name = video_name
        
    except Exception as e:
        print(f"\n❌ Error processing video: {e}")
        print("\nTroubleshooting tips:")
        print("- If GPU memory error: Restart runtime and try again")
        print("- If video format error: Convert video to MP4 format")

## 🔍 Step 6: Search Your Video!

Now you can search for content using natural language queries.

**Example queries:**
- "person walking"
- "black bag"
- "someone talking on phone"
- "car driving"
- "red shirt"

The system will return timestamps where that content appears!

In [ ]:
# Single search query
query = input("🔍 Enter your search query: ")

print(f"\nSearching for: '{query}'...")
print("=" * 60)

results = engine.search(
    query=query,
    top_k=5,
    similarity_threshold=0.5
)

if results:
    print(f"\n✅ Found {len(results)} results:\n")
    
    for i, result in enumerate(results, 1):
        print(f"{i}. ⏱️ Timestamp: {result['time_formatted']}")
        print(f"   📝 Caption: {result['caption']}")
        print(f"   📊 Confidence: {result['similarity_score']:.1%}")
        print(f"   🎥 Video: {result['video_name']}")
        print()
else:
    print("\n❌ No results found. Try:")
    print("   - Different search terms")
    print("   - More general queries")
    print("   - Lowering the similarity threshold")

## 🔎 Step 7: Batch Search (Multiple Queries)

Search for multiple things at once!

In [ ]:
# Define multiple queries
queries = [
    "person walking",
    "someone sitting",
    "black bag",
    "outdoor scene",
    "person talking"
]

print("🔍 Running batch search...\n")
print("=" * 60)

batch_results = engine.batch_search(queries, top_k=3)

for query, results in batch_results.items():
    print(f"\n📌 Query: '{query}'")
    print(f"   Found {len(results)} results")
    
    if results:
        for result in results[:2]:  # Show top 2
            print(f"   └─ {result['time_formatted']} - {result['caption'][:50]}... ({result['similarity_score']:.0%})")
    else:
        print("   └─ No results")

print("\n" + "=" * 60)

## 🎯 Step 8: Advanced Search with Filters

Search with additional filters:
- Filter by specific video
- Search within time range
- Adjust confidence threshold

In [ ]:
# Advanced search example
query = input("Enter search query: ")

# Optional: Filter by time window (in seconds)
use_time_filter = input("Filter by time range? (y/n): ").lower() == 'y'

time_window = None
if use_time_filter:
    start_time = float(input("Start time (seconds): "))
    end_time = float(input("End time (seconds): "))
    time_window = (start_time, end_time)

# Optional: Filter by video name
video_filter = None
if 'processed_video_name' in locals():
    filter_video = input(f"Search only in '{processed_video_name}'? (y/n): ").lower() == 'y'
    if filter_video:
        video_filter = processed_video_name

# Perform search
print(f"\n🔍 Searching with filters...")
results = engine.search(
    query=query,
    top_k=10,
    similarity_threshold=0.4,  # Lower threshold for more results
    video_filter=video_filter,
    time_window=time_window
)

print(f"\n✅ Found {len(results)} results:\n")
for i, result in enumerate(results, 1):
    print(f"{i}. {result['time_formatted']} - {result['caption'][:60]}... ({result['similarity_score']:.1%})")

## 📊 Step 9: View Database Statistics

Check the current state of your Pinecone database.

In [ ]:
import json

stats = engine.get_index_stats()

print("📊 PINECONE DATABASE STATISTICS")
print("=" * 60)
print(f"\n🗄️ Index Information:")
print(f"   Name: capstone")
print(f"   Total vectors: {stats.get('total_vectors', 0):,}")
print(f"   Dimension: {stats.get('dimension', 1024)}")
print(f"   Capacity mode: Serverless")
print(f"   Cloud: AWS")
print(f"   Region: us-east-1")

# Estimate storage
total_vectors = stats.get('total_vectors', 0)
estimated_mb = (total_vectors * 1024 * 4) / (1024 * 1024)  # 4 bytes per dimension
print(f"\n💾 Estimated storage: ~{estimated_mb:.1f} MB")

if 'namespaces' in stats:
    print(f"\n📁 Namespaces:")
    print(json.dumps(stats['namespaces'], indent=2))

## 🧹 Step 10: Cleanup (Optional)

**⚠️ Warning**: This will delete ALL vectors from your Pinecone index!

Only run this if you want to start fresh.

In [ ]:
# Uncomment to enable cleanup
# confirm = input("⚠️ Are you sure you want to delete ALL vectors? Type 'YES' to confirm: ")
# 
# if confirm == "YES":
#     print("🧹 Clearing database...")
#     engine.clear_index()
#     print("✅ Database cleared!")
# else:
#     print("❌ Cleanup cancelled")

print("⚠️ Cleanup is commented out for safety.")
print("Uncomment the code above if you need to clear the database.")

## 🎨 Step 11: Interactive Search Interface

Run this for a simple interactive search interface!

In [ ]:
print("🎯 INTERACTIVE VIDEO SEARCH")
print("=" * 60)
print("Enter your search queries (type 'quit' to exit)\n")

while True:
    query = input("\n🔍 Search: ").strip()
    
    if query.lower() in ['quit', 'exit', 'q']:
        print("\n👋 Goodbye!")
        break
    
    if not query:
        continue
    
    results = engine.search(query, top_k=5)
    
    if results:
        print(f"\n✅ Found {len(results)} results:")
        for i, result in enumerate(results, 1):
            score_emoji = "🟢" if result['similarity_score'] > 0.7 else "🟡" if result['similarity_score'] > 0.5 else "🟠"
            print(f"\n{i}. {score_emoji} {result['time_formatted']} ({result['similarity_score']:.0%})")
            print(f"   {result['caption']}")
    else:
        print("\n❌ No results found. Try a different query.")

---

## 🎉 You're All Set!

### What You Can Do Now:
1. ✅ Process more videos (repeat Step 4-5)
2. ✅ Search across all processed videos
3. ✅ Experiment with different queries
4. ✅ Adjust search parameters

### Tips for Best Results:
- Use descriptive queries ("person with red backpack" vs "person")
- Try different phrasings if you don't get results
- Lower the similarity threshold for more results
- Process multiple videos to build a searchable library

### Performance Notes:
- **GPU (T4)**: ~2-3 min per minute of video
- **CPU**: ~8-10 min per minute of video
- **Search**: <1 second per query

---

## 📚 Resources
- **GitHub Repo**: https://github.com/pranavacchu/capstone-BLIP.git
- **Pinecone Dashboard**: https://app.pinecone.io
- **BLIP Model**: Salesforce Research

---

**Made with ❤️ for video content search**